In [3]:
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import math
import torch
import torch.utils.data as data_utils
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import metrics
import pickle as pk
import sys
import time

In [4]:
train = pd.read_csv('train.csv')
split_idx = np.load('task_2_idx_split.npy', allow_pickle=True) # load pre-fixed data split indices
embs = np.load('fp_circ.npy') #load the fingerprints
y_names = train.columns[1:]

In [5]:
# use the first split for now
data = embs[split_idx[0][0],:]
test_idx = split_idx[0][1]
test_data = torch.Tensor(embs[test_idx,:])

In [6]:
# We removed searching for dropout rate based on the results
class Docking_Prediction(nn.Module):
    def __init__(self, layer_dims):
        super().__init__()        
        self.seq = []
        for i in range(len(layer_dims)-1):
            self.seq.append(nn.Linear(layer_dims[i], layer_dims[i+1]))
#            self.seq.append(nn.Dropout(dropout_rate, inplace=True))
            if i!=len(layer_dims)-2:
#                self.seq.append(nn.BatchNorm1d(layer_dims[i+1]))
                self.seq.append(nn.LeakyReLU())   
        self.sequential = nn.Sequential(*self.seq)
    
    def forward(self, x):
        return self.sequential(x)    

In [ ]:
def gen_layers(input_dim):    
    widths= range(1, input_dim+1, 256) #this sets the maximum number of layers
    n = np.random.randint(len(widths))
    if n==0:
        n+=1
    gap = np.floor(input_dim/n)
    layer_dims = [int(a*gap) for a in range(1,n)]
    
    if len(layer_dims)==1:
        return [input_dim, 1]
    else:
        layer_dims.append(input_dim)
        layer_dims.insert(0,1)
        return layer_dims[::-1]

In [ ]:
def gen_dropout():
    r = [0, 0.1, 0.2]
    n = np.random.randint(3)
    return r[n]

In [ ]:
emb_dim = embs.shape[1]
max_epoch = 100
batch_size = 64
max_iter = 10

In [ ]:
col=0 # Search the architecture for a column
labels = np.array(train[y_names[col]][split_idx[0][0]])
data_train, data_valid, label_train, label_valid=train_test_split(data, labels, test_size=0.2, random_state=42)

In [ ]:
mae = {}
L = []
#D = []
tic = time.time()
for ite in range(max_iter):
    layer_dims = gen_layers(emb_dim)
    dropout_rate = gen_dropout()
    
    L.append(layer_dims)
#    D.append(dropout_rate)
    print(ite, layer_dims)
    
    model = Docking_Prediction(layer_dims, dropout_rate)
    model_best = Docking_Prediction(layer_dims, dropout_rate)

    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.AdamW(model.parameters())
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, 
                                                          min_lr=1e-7, verbose=True)
    t=5
    patience = t
    loss_valid = []
    loss_train = []
    valid_error = 10e5
    e=1e-4
    error_best = 10e5


    for epoch in range(max_epoch):
        loss_run = 0
        
        n_batch = int(np.floor(data_train.shape[0]/batch_size))
        n_batch_valid = int(np.floor(data_valid.shape[0]/batch_size))

        for i in range(n_batch):
            data_batch= torch.Tensor(data_train[i*batch_size:(i+1)*batch_size,:])
            label_batch = torch.Tensor(label_train[i*batch_size:(i+1)*batch_size])

            outputs = model(data_batch)
            loss = criterion(torch.squeeze(outputs), label_batch)
            loss_run+=loss

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if i%1000==0:
                print(f'Batch({i+1}) loss: {loss}')

        # calculate validation loss
        loss_valid_batch = 0
        for i in range(n_batch_valid):

            label_batch = torch.Tensor(label_valid[i*batch_size:(i+1)*batch_size])
            data_batch= torch.Tensor(data_valid[i*batch_size:(i+1)*batch_size,:])

            outputs = model(data_batch)
            loss = criterion(torch.squeeze(outputs), label_batch)
            loss_valid_batch+=loss

        if loss_valid_batch/n_batch_valid >= error_best-e:
            patience-=1
        else:
            patience=t

        if patience == 0:
            break
        print('patience=', patience)

        valid_error=loss_valid_batch/n_batch_valid
        loss_valid.append(valid_error)

        if valid_error<error_best:
            model_best = model
            error_best = valid_error

        scheduler.step(valid_error)

        loss_run=loss_run/n_batch
        loss_train.append(loss_run)
        print('|epoch {:4d} | loss {:.6f} | valid loss {:.6f} |'.format(epoch, loss_run,valid_error)) 
    
    print('------------Model fitting finished-------------')
    
    # Model test
    preds_test = model_best(test_data)
    test_label = np.array(train[y_names[col]][test_idx])
    mae[ite] = metrics.mean_absolute_error(test_label, preds_test.detach().numpy())

    
    if min(mae, key=mae.get) == ite:
        torch.save(model_best.state_dict(), 'BestNN_circ/bestNN_circ_col{}'.format(col))
        print('!!!!!!!!!!!!!!saving best model!!!!!!!!!!!!!!')
        
pk.dump(L, open('layer_dims_col{}.pk'.format(col), 'wb'))
#pk.dump(D, open('dropout_col{}.pk'.format(col), 'wb'))

pk.dump(mae, open('mae_NN_circ_col{}.pk'.format(col), 'wb'))
toc=time.time()

In [ ]:
NN_arch = {}
for i in range(18):
    mae = pk.load('mae_NN_circ_col{}.pk'.format(i), 'rb')
    L = pk.load('layer_dims_col{}.pk'.format(col), 'wb')
    NN_arch[col] = L[min(mae, key=mae.get)]
pk.dump(NN_arch, open('NN_arch.pk','wb'))

# Model testing

In [7]:
NN_arch = pk.load(open('NN_arch.pk', 'rb'))

In [13]:
norm_mae=[]
for i in range(18):
    layer_dims = NN_arch[i]
    model = Docking_Prediction(layer_dims)
    model.load_state_dict(torch.load('BestNN_circ/bestNN_circ_col{}'.format(i)))#load the best model
    preds = model(test_data).detach().numpy()

    test_label=np.array(train[y_names[i]][test_idx])
    print(y_names[i], metrics.mean_absolute_error(test_label, preds))
    norm_mae.append(metrics.mean_absolute_error(test_label, preds)/(test_label.max()-test_label.min()))

3CLPro_pocket1 0.7455625983793647
ADRP-ADPR_pocket1 0.3468090994142603
ADRP-ADPR_pocket5 0.35057756812916857
ADRP_pocket1 0.2632114434953089
ADRP_pocket12 0.26126620909567233
ADRP_pocket13 0.29410418168085595
COV_pocket1 0.20552249665315506
COV_pocket2 0.20617000194566118
COV_pocket8 0.3156156268186922
COV_pocket10 0.31273062214992664
NSP9_pocket2 0.3541901033060638
NSP9_pocket7 0.28077987337492133
NSP15_pocket1 0.3129642783366309
ORF7A_pocket2 0.3016622075907389
PLPro_chainA_pocket3 0.3250212305640291
PLPro_chainA_pocket23 0.39383646656583854
PLPro_pocket6 0.2988809361086951
PLPro_pocket50 0.41308884387581446


In [19]:
norm_mae

[0.06545764691653773,
 0.029667159915676675,
 0.029141942487877687,
 0.035141714752377685,
 0.03483549454608965,
 0.028975781446389754,
 0.04326789403224317,
 0.042862786267289225,
 0.03364772140924224,
 0.027699789384404486,
 0.03335123383296269,
 0.028916567803802404,
 0.03161255336733645,
 0.03902486514757295,
 0.042709754344813285,
 0.037941856123876544,
 0.030498054704968885,
 0.033584458851692235]

In [17]:
i=0
layer_dims = NN_arch[i]
model = Docking_Prediction(layer_dims)
model.load_state_dict(torch.load('BestNN_circ/bestNN_circ_col{}'.format(i)))#load the best model
preds_test = model(test_data).detach().numpy()

In [ ]:
plt.figure(figsize=[6,6])
plt.plot(preds_test.detach().numpy(), np.array(train[y_names[col]][split_idx[0][1]]), '.')
plt.xlabel('Predicted score', fontsize=14)
plt.ylabel('True score', fontsize=14)
plt.savefig('col0.pdf')
plt.show()